In [21]:
import requests
import json
from config import api_key

In [24]:
with open('city.list.json', 'r') as ct:
    city_dict = json.load(ct)
    for x in city_dict:
    print(x['id'])

UnicodeDecodeError: 'cp949' codec can't decode byte 0xe2 in position 867: illegal multibyte sequence

In [22]:
city_list = []
for x in ct:
    city_list.append(x['id'])

TypeError: 'module' object is not iterable

In [ ]:
# api.openweathermap.org/data/2.5/weather?id={city id}&appid={your api key}
url = 'http://api.openweathermap.org/data/2.5/weather?'

base_url =